In [123]:
import glob
import os
import pandas as pd

In [124]:
# list of GSE IDs of processed ChIP-seq samples, or GSE IDs without any ChIP-seq samples
# this list is used to determine which HiChIP samples do not have matching ChIP-seq data
processed_gse_ids = ["GSE136090", "GSE154512", "GSE116193", "GSE105028", "GSE128106"]

gse_ids_without_matching_chipseq = ["GSE178598", "GSE101498", "GSE147854", "GSE151193", "GSE179755", "GSE173699"]
gse_ids_without_matching_chipseq.extend(["GSE92881", "GSE148691"])

#### Function to generate pandas dataframe

In [139]:
# Create a dataframe with the following columns:
# 1) Sample Name
# 2) HiC-Pro .allValidPairs file
# 3) Matching ChIP-seq peaks (if it exists)
# 4) Matching FitHiChIP peaks (if it has been processed yet)
# 5) Whether loop calling has been completed for this sample
# 6) Number of 5kb loops from HICCUPS
# 7) Number of 10kb loops from HICCUPS
# 8) Number of 25kb loops from HICCUPS

def generate_df():
    data = []
    
    hicpro_path = "/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_database/hichip-db-loop-calling/results/hicpro/"
    hicpro_path += "*/hic_results/data/*/*.allValidPairs"
    
    # iterate through each HiC-Pro output directory
    for fn in glob.glob(hicpro_path):
        
        sample_name = fn.split('/')[10]
        
        # set the path of the HiC-Pro output folder
        hicpro_output=fn
        
        chipseq_peak_file="Not processed or not available"
        chipline_path="/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_database/hichip-db-loop-calling/results/peaks"
        
        # check whether a folder matching sample_name exists in the merged_chipline folder
        merged_chipline_path = chipline_path + "/merged_chipline/" + sample_name
        if os.path.exists(merged_chipline_path):
            chipseq_peak_file=merged_chipline_path + "/FINAL_IDR_Peaks_FDR0.05.txt"
        
        # check whether a folder matching sample_name exists in the chipline folder
        chipline_path += "/chipline/" + sample_name + "/MACS2_Ext*/" + sample_name + ".macs2_peaks.narrowPeak_Q0.01filt"
        if chipseq_peak_file=="Not processed or not available" and len(glob.glob(chipline_path))>0:
            chipseq_peak_file = glob.glob(chipline_path)[0]
            
        # check if the matching ChIP-seq sample does not exist
        if any(gse_id in sample_name for gse_id in processed_chipseq_gse_ids):
            if not os.path.exists(chipseq_peak_file):
                chipseq_peak_file = "Not available"
            
        
        # check whether a folder matching the sample name exists in the fithichip folder
        fithichip_peak_file="Not processed"
        
        fithichip_peak_path="/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/fithichip/" + sample_name
        if os.path.exists(fithichip_peak_path):
            fithichip_peak_file = fithichip_peak_path + "/MACS2_ExtSize/out_macs2_peaks.narrowPeak"
            
        # check whether loop calling for this sample with FitHiChIP is complete
        loop_calling_complete = "No"
        fithichip_loops_path="/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/fithichip/" + sample_name
        if os.path.exists(fithichip_loops_path):
            loop_calling_complete = "Yes"
        
        
        # check the number of loops from HICCUPS at each resolution
        hiccups_loops_path="/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/hiccups/threshold_200_10kb_loops/" + sample_name
        
        hiccups_5000_numLoops="Not processed"
        hiccups_10000_numLoops="Not processed"
        hiccups_25000_numLoops="Not processed"
        if os.path.exists(hiccups_loops_path):
            hiccups_loops_5000_path = hiccups_loops_path + "/postprocessed_pixels_5000.bedpe"
            hiccups_loops_10000_path = hiccups_loops_path + "/postprocessed_pixels_10000.bedpe"
            hiccups_loops_25000_path = hiccups_loops_path + "/postprocessed_pixels_25000.bedpe"
            
            hiccups_5000_numLoops = sum(1 for line in open(hiccups_loops_5000_path)) - 2
            hiccups_10000_numLoops = sum(1 for line in open(hiccups_loops_10000_path)) - 2
            hiccups_25000_numLoops = sum(1 for line in open(hiccups_loops_25000_path)) - 2
        
        
        # add all the information for this sample to the 2D array
        sample_data = [sample_name, hicpro_output, chipseq_peak_file, fithichip_peak_file, loop_calling_complete]
        sample_data.extend([hiccups_5000_numLoops, hiccups_10000_numLoops, hiccups_25000_numLoops])
        
        data.append(sample_data)
        
    df = pd.DataFrame(data)
    return df
    

#### Display the pandas dataframe

In [140]:
# In the dataframe, "Not available" indicates that the necessary input to create the file does not exist
# "Not processed" indicates that the necessary input to create the file exists, but has not been processed yet

df = generate_df()

columns = ["Sample Name", "HiC-Pro Valid Pairs Path", "ChIP-seq Peaks Path", "FitHiChIP Peaks Path"]
columns.extend(["Loop Calling with FitHiChIP Complete?", "Number of Loops from HICCUPS (5kb)"])
columns.extend(["Number of Loops from HICCUPS (10kb)", "Number of Loops from HICCUPS (25kb)"])

df.columns = columns

df = df.reset_index(drop=True)
df.index += 2

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

,Sample Name,HiC-Pro Valid Pairs Path,ChIP-seq Peaks Path,FitHiChIP Peaks Path,Loop Calling with FitHiChIP Complete?,Number of Loops from HICCUPS (5kb),Number of Loops from HICCUPS (10kb),Number of Loops from HICCUPS (25kb)
2,BCBL1.GSE136090.Homo_Sapiens.H3K27ac.b1,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,Not processed,Not processed,Not processed
3,lgs000379.GSE116193.Homo_Sapiens.CTCF.b1,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,Not available,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,Not processed,Not processed,Not processed
4,lgs301283.GSE116193.Homo_Sapiens.CTCF.b1,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,Not available,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,Not processed,Not processed,Not processed
5,AoSMC.GSE178598.Homo_Sapiens.H3K27ac.b1,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,Not available,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,2999,4867,5199
6,H9-HS-KI.GSE105028.Homo_Sapiens.Rad21.b1,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,Not available,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,6291,14321,12906
7,HAVIC.GSE154512.Homo_Sapiens.H3K27ac.b1,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,Not processed,Not processed,Not processed
8,Th17.GSE101498.Homo_Sapiens.H3K27ac.b3,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,Not available,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,3030,5035,6853
9,GM.GSE101498.Homo_Sapiens.H3K27ac.b2,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,Not available,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,3549,5271,4383
10,Naive_Tcells.GSE101498.Homo_Sapiens.H3K27ac.b2,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,Not available,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,4215,6788,7495
11,BC3.GSE136090.Homo_Sapiens.H3K27ac.b1,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,/mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_databas...,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-l...,Yes,Not processed,Not processed,Not processed


#### Check the values for any specific row

In [141]:
row=36

print("Sample Name:", df["Sample Name"][row], "\n")
print("HiC-Pro Valid Pairs Path:", df["HiC-Pro Valid Pairs Path"][row], "\n")
print("ChIP-seq Peaks Path:", df["ChIP-seq Peaks Path"][row], "\n")
print("FitHiChIP Peaks Path:", df["FitHiChIP Peaks Path"][row], "\n")
print("Loop Calling with FitHiChIP Complete?:", df["Loop Calling with FitHiChIP Complete?"][row], "\n")
print("Number of Loops from HICCUPS (5kb):", df["Number of Loops from HICCUPS (5kb)"][row])
print("Number of Loops from HICCUPS (10kb):", df["Number of Loops from HICCUPS (10kb)"][row])
print("Number of Loops from HICCUPS (25kb):", df["Number of Loops from HICCUPS (25kb)"][row], "\n")


Sample Name: 293T.GSE128106.Homo_Sapiens.YY1.b1 

HiC-Pro Valid Pairs Path: /mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_database/hichip-db-loop-calling/results/hicpro/293T.GSE128106.Homo_Sapiens.YY1.b1/hic_results/data/293T.GSE128106.Homo_Sapiens.YY1.b1/293T.GSE128106.Homo_Sapiens.YY1.b1.allValidPairs 

ChIP-seq Peaks Path: /mnt/BioAdHoc/Groups/vd-ay/nrao/hichip_database/hichip-db-loop-calling/results/peaks/merged_chipline/293T.GSE128106.Homo_Sapiens.YY1.b1/FINAL_IDR_Peaks_FDR0.05.txt 

FitHiChIP Peaks Path: /mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/fithichip/293T.GSE128106.Homo_Sapiens.YY1.b1/MACS2_ExtSize/out_macs2_peaks.narrowPeak 

Loop Calling with FitHiChIP Complete?: Yes 

Number of Loops from HICCUPS (5kb): Not processed
Number of Loops from HICCUPS (10kb): Not processed
Number of Loops from HICCUPS (25kb): Not processed 



#### Output the dataframe as a tsv file

In [142]:
df2 = df.copy(deep=True)
df2.set_index("Sample Name", inplace=True)

# set path of file
tsv_path = "../../../results/samplesheets/loop_calling_tracker.tsv"
df2.to_csv(tsv_path, sep="\t")